In [3]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, chi2

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from bs4 import BeautifulSoup
from langdetect import detect
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Pre Processing

In [4]:
df = pd.read_csv("../Dataset/IMDB Dataset.csv")
df.head()
df = df.rename(columns={'review': 'OriginalReviews'})
df = df.rename(columns={'sentiment': 'OutputSentiment'})
df_subset = df.sample(n=5000).reset_index(drop=True)
df_subset.head()
df_subset['OutputSentiment'].value_counts()

OutputSentiment
negative    2532
positive    2468
Name: count, dtype: int64

In [5]:
# Function to remove HTML tags from a text
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

#Stopwords removal
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from a text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Function to remove URLs from a text
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)

# Function to filter non-English comments
def filter_non_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:

# Remove numbers from the 'reviews' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('\d+', '')

#Remove punctuations
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('[{}]'.format(string.punctuation), '')

# Apply the remove_stopwords function to the 'reviews' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_stopwords)

# Apply the remove_html_tags function to the 'text' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_html_tags)

# Apply the remove_urls function to the 'reviews' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_urls)

# Create a boolean mask for non-English reviews
mask = df_subset['OriginalReviews'].apply(filter_non_english)

# Create a new DataFrame containing only English reviews
df_subset = df_subset[mask]

nltk.download('wordnet')
# Lemmatization function
def lemmatize_column(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized_words)

# Apply the function to the specific column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(lemmatize_column)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sdidd\AppData\Local\Temp\ipykernel_21296\3669464969.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('\d+', '')
C:\Users\sdidd\AppData\Local\Temp\ipykernel_21296\3685019402.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
df_subset.to_csv("../csv/Preprocessed_data.csv",index=False)

## Feature Extraction Using TF-IDF

In [8]:
preprocessed = pd.read_csv('../csv/Preprocessed_data.csv')

In [48]:
num_features_to_keep = 13000

# Create a pipeline with TfidfVectorizer and SelectKBest
pipeline = make_pipeline(TfidfVectorizer(), SelectKBest(f_classif, k=num_features_to_keep))

# Fit and transform your data
X_transformed = pipeline.fit_transform(preprocessed['OriginalReviews'], preprocessed['OutputSentiment'])

# Get the selected feature names
selected_feature_names = pipeline.named_steps['tfidfvectorizer'].get_feature_names_out()[pipeline.named_steps['selectkbest'].get_support()]

# Create a DataFrame with the selected features
selected_features_df = pd.DataFrame(X_transformed.toarray(), columns=selected_feature_names)

# Concatenate the existing DataFrame with the new selected features DataFrame
tfidf_df_13k = pd.concat([preprocessed, selected_features_df], axis=1)

tfidf_df_13k.head()

tfidf_df_13k.to_csv("../csv/tfidf_df_13k.csv")

## CONNOTATIONS

In [49]:
# Download the VADER lexicon (run this once)
nltk.download('vader_lexicon')
delimiter = '\t'

# Read the text file into a DataFrame
positive = pd.read_csv(r'..\Connotations\positive-words.txt', sep=delimiter, names=['words'])
negative = pd.read_csv(r'..\Connotations\negative-words.txt', sep=delimiter, names=['words'])
connotations = pd.read_csv(r"..\Connotations\connotations.csv")

word_emotion_map = dict(zip(connotations['word'], connotations['emotion']))

def update_counts(review):
    positive_count = sum(1 for word in review.split() if word in word_emotion_map and word_emotion_map[word] == 'positive')
    negative_count = sum(1 for word in review.split() if word in word_emotion_map and word_emotion_map[word] == 'negative')
    return positive_count, negative_count

tfidf_df_13k[['Positive_Connotation_Count', 'Negative_Connotation_Count']] = tfidf_df_13k['OriginalReviews'].apply(update_counts).tolist()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [50]:
# Load positive and negative words from files
positive_words_df = pd.read_csv(r'..\Connotations\positive-words.txt', header=None, names=['words'])
negative_words_df = pd.read_csv(r'..\Connotations\negative-words.txt', header=None, names=['words'])

# Convert DataFrame columns to sets
positive_words = set(positive_words_df['words'].tolist())
negative_words = set(negative_words_df['words'].tolist())

# Assuming 'tfidf_df_13k' is your DataFrame
# Define a function to update counts based on positive and negative words
def update_word_counts(review):
    positive_count = sum(1 for word in review.split() if word in positive_words)
    negative_count = sum(1 for word in review.split() if word in negative_words)
    return positive_count, negative_count

# Apply the function to the 'OriginalReviews' column and unpack the result into two new columns
tfidf_df_13k[['Positive_Word_Count', 'Negative_Word_Count']] = tfidf_df_13k['OriginalReviews'].apply(update_word_counts).tolist()

In [51]:
from nltk.sentiment import SentimentIntensityAnalyzer
# Use VADER for sentiment analysis
sid = SentimentIntensityAnalyzer()

def vader_sentiment(review):
    scores = sid.polarity_scores(review)
    return scores['pos'] *100, scores['neg'] * 100

# Apply the function to the 'OriginalReviews' column and unpack the result into two new columns
tfidf_df_13k[['Positive_VADER_Count', 'Negative_VADER_Count']] = tfidf_df_13k['OriginalReviews'].apply(vader_sentiment).tolist()

tfidf_df_13k.to_csv("../csv/tfidf_df_13k_connotations_vader.csv")

In [52]:
tfidf_df_13k

,OriginalReviews,OutputSentiment,00,000,00001,0080,0083,00am,01,06,...,âme,æsthetic,ís,ünel,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,"seeing excellent film 100 time , still find ne...",positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17,11,8,3,47.2,0.0
1,Wicked Little thing really good movie.I say pa...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22,8,7,3,28.7,7.4
2,"avid Gone Wind fan , disappointed watch origin...",negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,63,38,10,9,22.4,11.9
3,"Well , TiVo recorded Angelina Jolie . 2.5 star...",negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,25,9,10,17.2,13.0
4,find almost touching Star Trek fan try despera...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,59,69,10,26,16.5,21.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,A-team still repeating every day Dutch televis...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,29,11,7,3,28.1,14.2
4994,William Castle notorious among horror fan B-gr...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,93,45,23,10,23.8,6.2
4995,took 4 year old twin see movie today would rec...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,16,17,2,6,7.8,31.0
4996,checked one DVD dollar could easily smile drea...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,19,25,3,12,15.2,27.2


In [53]:
tfidf_df_13k = pd.read_csv("../csv/tfidf_df_13k.csv")

In [54]:
tfidf_df_13k_connotations = pd.read_csv('../csv/tfidf_df_13k_connotations_vader.csv')

In [55]:
tfidf_df_13k_connotations = tfidf_df_13k_connotations.drop('Unnamed: 0',axis=1)

In [56]:
tfidf_df_13k_connotations

,OriginalReviews,OutputSentiment,00,000,00001,0080,0083,00am,01,06,...,âme,æsthetic,ís,ünel,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,"seeing excellent film 100 time , still find ne...",positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17,11,8,3,47.2,0.0
1,Wicked Little thing really good movie.I say pa...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22,8,7,3,28.7,7.4
2,"avid Gone Wind fan , disappointed watch origin...",negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,63,38,10,9,22.4,11.9
3,"Well , TiVo recorded Angelina Jolie . 2.5 star...",negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,25,9,10,17.2,13.0
4,find almost touching Star Trek fan try despera...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,59,69,10,26,16.5,21.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,A-team still repeating every day Dutch televis...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,29,11,7,3,28.1,14.2
4994,William Castle notorious among horror fan B-gr...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,93,45,23,10,23.8,6.2
4995,took 4 year old twin see movie today would rec...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,16,17,2,6,7.8,31.0
4996,checked one DVD dollar could easily smile drea...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,19,25,3,12,15.2,27.2


In [57]:
df_statistical = tfidf_df_13k_connotations.drop(columns=['OriginalReviews','Positive_Connotation_Count','Negative_Connotation_Count','Positive_Word_Count','Negative_Word_Count','Positive_VADER_Count','Negative_VADER_Count'], axis=1)
df_statistical.head()

,OutputSentiment,00,000,00001,0080,0083,00am,01,06,07,...,zula,zuleika,zwick,zázvorková,ànd,ángela,âme,æsthetic,ís,ünel
0,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
label = LabelEncoder()
df_statistical['OutputSentiment'] = label.fit_transform(df_statistical['OutputSentiment'])

## CHI SQAURE

In [69]:
# This will get the top 5000 relavant features out of the sample
chi2_selector = SelectKBest(chi2, k=5000)

# This will transform the dataset i.e, it will reduce the dimensions by just considering the relavant features only
X = df_statistical.drop(columns=['OutputSentiment'])
y = df_statistical['OutputSentiment']
X_5000 = chi2_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X.columns[selected_feature_indices]

chisq_5k = X[selected_feature_names]
chisq_5k.head()

chisq_5k = pd.concat([chisq_5k,tfidf_df_13k_connotations.iloc[:, -6:]],axis=1)
chisq_5k.head()

,00,000,00001,01,102,11th,13th,146,17,180,...,zombie,zp,zu,zucher,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17,11,8,3,47.2,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22,8,7,3,28.7,7.4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,63,38,10,9,22.4,11.9
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,25,9,10,17.2,13.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,59,69,10,26,16.5,21.4


In [70]:
# This will get the top 5000 relavant features out of the sample
chi2_selector = SelectKBest(chi2, k=8000)

# This will transform the dataset i.e, it will reduce the dimensions by just considering the relavant features only
X = df_statistical.drop(columns=['OutputSentiment'])
y = df_statistical['OutputSentiment']
X_8000 = chi2_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X.columns[selected_feature_indices]

chisq_8k = X[selected_feature_names]
chisq_8k.head()

chisq_8k = pd.concat([chisq_8k,tfidf_df_13k_connotations.iloc[:, -6:]],axis=1)
chisq_8k.head()

,00,000,00001,00am,01,07,10,1000,102,11,...,zp,zu,zucher,âme,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17,11,8,3,47.2,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22,8,7,3,28.7,7.4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,63,38,10,9,22.4,11.9
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,25,9,10,17.2,13.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,59,69,10,26,16.5,21.4


In [71]:
chisq_8k

,00,000,00001,00am,01,07,10,1000,102,11,...,zp,zu,zucher,âme,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17,11,8,3,47.2,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22,8,7,3,28.7,7.4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,63,38,10,9,22.4,11.9
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31,25,9,10,17.2,13.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,59,69,10,26,16.5,21.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,0.0,0.0,0.0,0.0,0.0,0.0,0.061638,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,29,11,7,3,28.1,14.2
4994,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,93,45,23,10,23.8,6.2
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,16,17,2,6,7.8,31.0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,19,25,3,12,15.2,27.2


## CHI SQAURE CLASSIFICATION

In [72]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

# Multinomial Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_scores = cross_val_score(nb_classifier, chisq_8k, y, cv=5)

print("Multinomial Naive Bayes Cross-Validation Scores:")
print(nb_scores)
print("Mean Accuracy:", np.mean(nb_scores))

# k-Nearest Neighbors Classifier
knn_classifier = KNeighborsClassifier()
knn_scores = cross_val_score(knn_classifier, chisq_5k, y, cv=5)

print("\nk-Nearest Neighbors Cross-Validation Scores:")
print(knn_scores)
print("Mean Accuracy:", np.mean(knn_scores))

Multinomial Naive Bayes Cross-Validation Scores:
[0.751      0.751      0.765      0.73373373 0.76676677]
Mean Accuracy: 0.7535001001001002

k-Nearest Neighbors Cross-Validation Scores:
[0.701      0.691      0.697      0.7027027  0.69469469]
Mean Accuracy: 0.6972794794794794


In [73]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Load your data
# Assuming X and y are your features and target variables

# Initialize models
svm_model = SVC(kernel='linear')  # Linear SVM
logistic_model = LogisticRegression()

# Initialize KFold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform 5-fold cross-validation for SVM
svm_scores = cross_val_score(svm_model, chisq_5k, y, cv=kfold)

# Perform 5-fold cross-validation for Logistic Regression
logistic_scores = cross_val_score(logistic_model, chisq_8k, y, cv=kfold)

# Display the cross-validation scores
print("SVM Cross-validation scores:", svm_scores)
print("Logistic Regression Cross-validation scores:", logistic_scores)

# Optionally, you can calculate mean and standard deviation of the scores
print("SVM Mean Accuracy:", np.mean(svm_scores))
print("SVM Standard Deviation of Accuracy:", np.std(svm_scores))
print("Logistic Regression Mean Accuracy:", np.mean(logistic_scores))
print("Logistic Regression Standard Deviation of Accuracy:", np.std(logistic_scores))

c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

SVM Cross-validation scores: [0.893      0.874      0.895      0.87087087 0.87087087]
Logistic Regression Cross-validation scores: [0.857      0.825      0.834      0.82582583 0.82282282]
SVM Mean Accuracy: 0.8807483483483484
SVM Standard Deviation of Accuracy: 0.010898457423509526
Logistic Regression Mean Accuracy: 0.8329297297297298
Logistic Regression Standard Deviation of Accuracy: 0.012617598985723264


c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
